# Load the tests to compare

In [1]:
import os
import sys
import json
import pickle
import pandas as pd
__file__ = globals()['_dh'][0]

# get an absolute path to the directory that contains parent files
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, '..', '..')))

from experiments.utils.constants import TESTS_RESULTS_PATH
series = 1
start_test = 0
end_test = 3
tests_ids = [i for i in range(start_test, end_test)]
tests_names = [i for i in range(start_test, end_test)]

tests = {}


for test_id, test_name in zip(tests_ids, tests_names):
    test_path = os.path.join(
        TESTS_RESULTS_PATH,
        'series',
        str(series),
        'tests',
        str(test_id))
    with open(os.path.join(test_path, 'info.json')) as cf:
        info = json.loads(cf.read())
    with open(os.path.join(test_path, 'episodes.pickle'), 'rb') as input_file:
        episodes = pickle.load(input_file)
    tests.update({test_name: {
        'info': info,
        'episodes': episodes
    }})
test_types = [test['info']['algorithm'] for _, test in tests.items()]
print("availeble tests:\n {}\ntests_types:\n {}".format(
    list(tests.keys()),
    test_types))

availeble tests:
 [0, 1, 2]
tests_types:
 ['PPO', 'PPO', 'PPO']


# All Stats of a test

In [2]:
test_id = 0
episdoe_id = 0
print(f'tests entry structure:\n{tests[test_id].keys()}')
print(f"tests entry info structure:\n{tests[test_id]['info'].keys()}")
print(f"tests entry episodes info:\n{tests[test_id]['episodes'][episdoe_id].columns}")

tests entry structure:
dict_keys(['info', 'episodes'])
tests entry info structure:
dict_keys(['type_env', 'series', 'cluster_id', 'workload_id', 'checkpoint', 'experiment_str', 'experiments', 'episode_length', 'num_episodes', 'algorithm', 'penalty_consolidated', 'num_workers'])
tests entry episodes info:
Index(['action', 'num_consolidated', 'num_overloaded', 'scheduling_timestep',
       'scheduling_success', 'reward_illegal', 'reward_u', 'reward_c',
       'reward_v', 'reward_g', 'reward_p', 'reward'],
      dtype='object')


# Extracting necessary stats of all tests

In [3]:
final_stats = []
for test_id in range(start_test, end_test):
    # TODO check for correctness
    num_consolidated_episodes_average = [episode['num_consolidated'].mean() for episode in tests[test_id]['episodes']]
    num_consolidated_test_average = sum(num_consolidated_episodes_average) / len(num_consolidated_episodes_average)

    num_overloaded_episodes_average = [episode['num_overloaded'].mean() for episode in tests[test_id]['episodes']]
    num_overloaded_test_average = sum(num_overloaded_episodes_average) / len(num_overloaded_episodes_average)

    scheduling_success_episodes_average = [episode['scheduling_success'].mean() for episode in tests[test_id]['episodes']]
    scheduling_success_test_average = sum(scheduling_success_episodes_average) / len(scheduling_success_episodes_average)

    reward_illegal_episodes_average = [episode['reward_illegal'].mean() for episode in tests[test_id]['episodes']]
    reward_illegal_test_average = sum(reward_illegal_episodes_average) / len(reward_illegal_episodes_average)

    reward_u_episodes_average = [episode['reward_u'].mean() for episode in tests[test_id]['episodes']]
    reward_u_test_average = sum(reward_u_episodes_average) / len(reward_u_episodes_average)

    reward_c_episodes_average = [episode['reward_c'].mean() for episode in tests[test_id]['episodes']]
    reward_c_test_average = sum(reward_c_episodes_average) / len(reward_c_episodes_average)

    reward_v_episodes_average = [episode['reward_v'].mean() for episode in tests[test_id]['episodes']]
    reward_v_test_average = sum(reward_v_episodes_average) / len(reward_v_episodes_average)

    reward_g_episodes_average = [episode['reward_g'].mean() for episode in tests[test_id]['episodes']]
    reward_g_test_average = sum(reward_g_episodes_average) / len(reward_g_episodes_average)

    reward_p_episodes_average = [episode['reward_p'].mean() for episode in tests[test_id]['episodes']]
    reward_p_test_average = sum(reward_p_episodes_average) / len(reward_p_episodes_average)

    reward_episodes_average = [episode['reward'].mean() for episode in tests[test_id]['episodes']]
    reward_test_average = sum(reward_episodes_average) / len(reward_episodes_average)


    final_stats.append({
        'test_id': test_id,
        'checkpoint': tests[test_id]['info']['checkpoint'],
        # TODO where is workload id
        'workload_id': tests[test_id]['info']['workload_id'],
        'experiments': tests[test_id]['info']['experiments'],
        'episode_length': tests[test_id]['info']['episode_length'],
        'num_episodes': tests[test_id]['info']['num_episodes'],
        'penalalty_consolidated': tests[test_id]['info']['penalty_consolidated'],
        'num_consolidated_test_average': num_consolidated_test_average
    })

final_stats_df = pd.DataFrame(final_stats)
final_stats_df['test_types'] = test_types
# final_stats_df.loc[df['penalalty_consolidated']]

# TODO add the following information to the table (from the info.json file)
# - penatly_latency
# - penalty_conslidation

In [4]:
test_id = 0
tests[test_id]['info']

{'type_env': 'sim-scheduler',
 'series': 1,
 'cluster_id': 0,
 'workload_id': 0,
 'checkpoint': 100,
 'experiment_str': 'PPO_SimSchedulerEnv_744f8_00000_0_2022-06-11_18-03-45',
 'experiments': 0,
 'episode_length': 10,
 'num_episodes': 1,
 'algorithm': 'PPO',
 'penalty_consolidated': 1,
 'num_workers': 4}

# compare tests - tables

In [5]:
final_stats_df

,test_id,checkpoint,workload_id,experiments,episode_length,num_episodes,penalalty_consolidated,num_consolidated_test_average,test_types
0,0,100,0,0,10,1,1,2.5000,PPO
1,1,100,0,0,200,1,1,2.0625,PPO
2,2,100,0,0,1000,1,1,2.0625,PPO


In [39]:
final_stats_df[
    # (final_stats_df['network_id'] == 1) &\
        (final_stats_df['num_consolidated_test_average'] == 2.5)]

,test_id,checkpoint,workload_id,experiments,episode_length,num_episodes,penalalty_consolidated,num_consolidated_test_average,test_types
0,0,100,0,0,10,1,1,2.5,PPO


# compare tests - plots

In [41]:
# TODO decide about plots and then make it based on https://github.com/saeid93/mobile-kube/blob/main/experiments/analysis/analysis_test_70-(final).ipynb